In [1]:
import os
import json
import re
from bs4 import BeautifulSoup


import pandas as pd
import numpy as np

from collections import namedtuple

import warnings
warnings.filterwarnings('ignore')

In [2]:
DATA_DIR = '../../data/'

data = pd.read_csv(os.path.join(DATA_DIR, 'places.csv'))
print(data.shape)
print(data.columns)
data

(1, 31)
Index(['vicinity', 'formatted_address', 'id', 'international_phone_number',
       'adr_address', 'url', 'photos', 'icon', 'name', 'types', 'rating',
       'utc_offset', 'geometry_viewport_southwest_lat', 'reviews',
       'price_level', 'geometry_viewport_northeast_lat', 'scope', 'place_id',
       'plus_code_global_code', 'formatted_phone_number',
       'plus_code_compound_code', 'geometry_location_lng',
       'geometry_viewport_northeast_lng', 'opening_hours_weekday_text',
       'website', 'geometry_location_lat', 'opening_hours_open_now',
       'address_components', 'geometry_viewport_southwest_lng', 'reference',
       'opening_hours_periods'],
      dtype='object')


,vicinity,formatted_address,id,international_phone_number,adr_address,url,photos,icon,name,types,...,geometry_location_lng,geometry_viewport_northeast_lng,opening_hours_weekday_text,website,geometry_location_lat,opening_hours_open_now,address_components,geometry_viewport_southwest_lng,reference,opening_hours_periods
0,"['Vatslava Havela Boulevard, 9В, Kyiv']","['Vatslava Havela Boulevard, 9В, Kyiv, Ukraine...",['24ef24ecd5d9911c1972e8aa43429541ad0f2f70'],['+380 44 300 0330'],"['<span class=""street-address"">Vatslava Havela...",['https://maps.google.com/?cid=161885582897466...,"['[{\'height\': 2736, \'html_attributions\': [...",['https://maps.gstatic.com/mapfiles/place_api/...,['Orly Park'],"[""['restaurant', 'point_of_interest', 'food', ...",...,[30.4265211],[30.42900149999999],"[""['Monday: Open 24 hours', 'Tuesday: Open 24 ...",['http://orlypark.com.ua/'],[50.4440215],[True],"['[{\'long_name\': \'9В\', \'short_name\': \'9...",[30.42499190000001],['ChIJBXnk-RbM1EARU2oObv5PqeA'],"[""[{'open': {'day': 0, 'time': '0000'}}]""]"


In [4]:
data[[
    'geometry_location_lng', 'geometry_viewport_southwest_lat',
    'geometry_viewport_northeast_lng', 'geometry_location_lat',
    'geometry_viewport_southwest_lng', 'geometry_viewport_northeast_lat',
]]

,geometry_location_lng,geometry_viewport_southwest_lat,geometry_viewport_northeast_lng,geometry_location_lat,geometry_viewport_southwest_lng,geometry_viewport_northeast_lat
0,[30.4265211],[50.4429178697085],[30.42900149999999],[50.4440215],[30.42499190000001],[50.44561583029149]


In [11]:
data['reference'].values[0]

"['ChIJBXnk-RbM1EARU2oObv5PqeA']"

# Transfrom df to DB_Places

In [87]:
get_value = lambda x: x if type(x) is not list else x[0]

def get_address(serie):
    addresses = np.array([None] * len(serie))
    cities = np.array([None] * len(serie))
    for index, row in enumerate(serie):
        soup = BeautifulSoup(row)
        addresses[index] = soup.find('span', class_='street-address').text
        cities[index] = soup.find('span', class_='locality').text
    return addresses, cities


In [86]:
data['reference'].apply(get_value)

0    ['ChIJBXnk-RbM1EARU2oObv5PqeA']
Name: reference, dtype: object

In [64]:
results = dict()
results['name'] = data['name'].apply(get_value)
results['website'] = data['website'].apply(get_value)
results['place_id'] = data['reference'].apply(get_value)
results['loc_x'] = data['geometry_location_lng'].apply(get_value)
results['loc_y'] = data['geometry_location_lat'].apply(get_value)
results['phone'] = data['international_phone_number'].apply(get_value)

results['address'], results['city'] = get_address(data['adr_address'])

In [74]:
for k, v in results.items():
    print('{k}:\t{v}'.format(k=k, v=v))

place_id:	0    ['ChIJBXnk-RbM1EARU2oObv5PqeA']
Name: reference, dtype: object
address:	['Vatslava Havela Boulevard, 9В']
phone:	0    ['+380 44 300 0330']
Name: international_phone_number, dtype: object
loc_x:	0    [30.4265211]
Name: geometry_location_lng, dtype: object
name:	0    ['Orly Park']
Name: name, dtype: object
loc_y:	0    [50.4440215]
Name: geometry_location_lat, dtype: object
website:	0    ['http://orlypark.com.ua/']
Name: website, dtype: object
city:	['Kyiv']


In [88]:
pd.DataFrame(results)

,address,city,loc_x,loc_y,name,phone,place_id,website
0,"Vatslava Havela Boulevard, 9В",Kyiv,[30.4265211],[50.4440215],['Orly Park'],['+380 44 300 0330'],['ChIJBXnk-RbM1EARU2oObv5PqeA'],['http://orlypark.com.ua/']


In [98]:
data['opening_hours_periods'][0]

'["[{\'open\': {\'day\': 0, \'time\': \'0000\'}}]"]'

In [127]:
print(data['reviews'][0])

["[{'author_name': 'Yevgen Chupak', 'author_url': 'https://www.google.com/maps/contrib/115359845398019720660/reviews', 'language': 'en', 'profile_photo_url': 'https://lh5.googleusercontent.com/-pcjZgo8Fedo/AAAAAAAAAAI/AAAAAAAALMw/jzvjbqcty2E/s128-c0x00000000-cc-rp-mo-ba4/photo.jpg', 'rating': 3, 'relative_time_description': '2 weeks ago', 'text': 'Nice place to walk. Not much to see and to do', 'time': 1542439583}, {'author_name': 'Orçun MERT', 'author_url': 'https://www.google.com/maps/contrib/116097876146427007195/reviews', 'language': 'en', 'profile_photo_url': 'https://lh6.googleusercontent.com/-cKMFuQoKqYs/AAAAAAAAAAI/AAAAAAAAB4M/g2XOlTFghUk/s128-c0x00000000-cc-rp-mo-ba3/photo.jpg', 'rating': 5, 'relative_time_description': '6 months ago', 'text': 'We like the hotel. 10 km to city center  but we have a  rented car ao its very easy to us. The rooms are new large and very clean. The price is reasonable', 'time': 1525789433}, {'author_name': 'Shlomi Krasner', 'author_url': 'https://w

In [240]:
def list_parser(query):
    pattern = r'^\["(?P<list_>.*)"\]$'
    result = re.search(pattern, query)
    if result:
#         return result.group('list_')
        return json.loads(
            '{"text": ' + result.group('list_').replace('\'', '"') + ' , "status" : "OK"}', encoding='utf-8'
        )
    return result

In [242]:
obj = list_parser(data["reviews"][0])

In [245]:
obj['text'][0]['author_name']

'Yevgen Chupak'

# Working Time

In [21]:
data1 = pd.read_csv(os.path.join(DATA_DIR, 'khreschatyk_new.csv'))
print(data1.shape)
print(data1.columns)
data1

(2, 31)
Index(['opening_hours_open_now', 'scope', 'reviews', 'types',
       'plus_code_global_code', 'geometry_viewport_southwest_lng',
       'plus_code_compound_code', 'address_components', 'icon', 'place_id',
       'id', 'rating', 'photos', 'international_phone_number', 'url',
       'opening_hours_weekday_text', 'geometry_location_lng',
       'geometry_viewport_southwest_lat', 'geometry_viewport_northeast_lat',
       'opening_hours_periods', 'formatted_address', 'formatted_phone_number',
       'utc_offset', 'geometry_location_lat', 'adr_address', 'website',
       'price_level', 'vicinity', 'reference',
       'geometry_viewport_northeast_lng', 'name'],
      dtype='object')


,opening_hours_open_now,scope,reviews,types,plus_code_global_code,geometry_viewport_southwest_lng,plus_code_compound_code,address_components,icon,place_id,...,formatted_phone_number,utc_offset,geometry_location_lat,adr_address,website,price_level,vicinity,reference,geometry_viewport_northeast_lng,name
0,['True'],['GOOGLE'],"[""[{'author_name': 'Yevgen Chupak', 'author_ur...","[""['restaurant', 'point_of_interest', 'food', ...",['9G2GCCVG+JJ'],['30.42499190000001'],"['CCVG+JJ Kyiv, Kyiv city, Ukraine']","['[{\'long_name\': \'9В\', \'short_name\': \'9...",['https://maps.gstatic.com/mapfiles/place_api/...,['ChIJBXnk-RbM1EARU2oObv5PqeA'],...,['044 300 0330'],['120'],['50.4440215'],"['<span class=""street-address"">Vatslava Havela...",['http://orlypark.com.ua/'],['3'],"['Vatslava Havela Boulevard, 9В, Kyiv']",['ChIJBXnk-RbM1EARU2oObv5PqeA'],['30.42900149999999'],['Orly Park']
1,['True'],['GOOGLE'],"['[{\'author_name\': \'Matthew Forster\', \'au...","[""['restaurant', 'point_of_interest', 'food', ...",['9G2GFGC7+2R'],['30.5132813697085'],"['FGC7+2R Kyiv, Kyiv city, Ukraine']","['[{\'long_name\': \'35/35\', \'short_name\': ...",['https://maps.gstatic.com/mapfiles/place_api/...,['ChIJ43dtiRTO1EAROneSaINAGXE'],...,['096 424 2424'],['120'],['50.470022'],"['<span class=""street-address"">Voloska St, 35/...",['http://www.sushi24.ua/'],['2'],"['Voloska Street, 35/35, Kyiv']",['ChIJ43dtiRTO1EAROneSaINAGXE'],['30.5159793302915'],['Sushi 24']


In [22]:
data1[[
    'opening_hours_weekday_text',
    'opening_hours_open_now',
    'opening_hours_periods'
]]

,opening_hours_weekday_text,opening_hours_open_now,opening_hours_periods
0,"[""['Monday: Open 24 hours', 'Tuesday: Open 24 ...",['True'],"[""[{'open': {'day': 0, 'time': '0000'}}]""]"
1,"[""['Monday: 10:00 AM – 3:00 AM', 'Tuesday: 10:...",['True'],"[""[{'close': {'day': 1, 'time': '0300'}, 'open..."


In [30]:
print(data1.opening_hours_weekday_text[0])
print('-' * 100)
print(data1.opening_hours_weekday_text[1])

["['Monday: Open 24 hours', 'Tuesday: Open 24 hours', 'Wednesday: Open 24 hours', 'Thursday: Open 24 hours', 'Friday: Open 24 hours', 'Saturday: Open 24 hours', 'Sunday: Open 24 hours']"]
----------------------------------------------------------------------------------------------------
["['Monday: 10:00 AM – 3:00 AM', 'Tuesday: 10:00 AM – 3:00 AM', 'Wednesday: 10:00 AM – 3:00 AM', 'Thursday: 10:00 AM – 3:00 AM', 'Friday: 10:00 AM – 3:00 AM', 'Saturday: 10:00 AM – 3:00 AM', 'Sunday: 10:00 AM – 3:00 AM']"]
